In [116]:
import numpy as np
'''
Prim 算法, 构建最小生成树

Prim 算法基本思路: 
    设一个图 G=<V, E>.
    设最小生成树所包含的边的集合 MST. 最开始MST=空集.
    维护一个集合 V_A, 往 V_A 中不断增加结点.
    每次选出连接 V_A 和 V-V_A 中顶点的权值最小的边, 把这条边加入到 MST 中.
    并且把这条边在 V-V_A 中的顶点加到 V_A 中.
    这样重复|V|-1 次, 找出|V|-1 条符合上述迭代方法的边, 就构成了最小生成树.

Prim 算法实现一: 
    见下面 Prim1 函数.
    设两个数组 min_wei 和 pred, 其中:
    min_wei[u]=w 表示 V_A 中的结点到 u 的最短边权值, 其中 u 属于 V-V_A
    pred[u]=v 表示上述 min_wei[u]=w 取到的时候, V_A 中与 u 相连的结点.

    每次通过排序, 维护 min_wei, pred, 进而找出权值最小的, 横跨 V_A, V-V_A 的边.
    把这条边(v, u) (v 属于 V_A, u 属于 V-V_A)中的 u 加入到 V_A 中, 然后如此迭代.
    并且把这条边加入 MST.

    如此重复|V|-1 次, 可以得到 MST.

    复杂度估计:
        主要复杂度花在找最小权值的横跨 V_A, V-V_A 的边上了. 复杂度大致在 O(|V|^3)左右?
        (prim1 函数的 for 循环中嵌入一个 update, update 函数大概是 O(|V|^2))
'''

'\nPrim 算法, 构建最小生成树\n\nPrim 算法基本思路: \n    设一个图 G=<V, E>.\n    设最小生成树所包含的边的集合 MST. 最开始MST=空集.\n    维护一个集合 V_A, 往 V_A 中不断增加结点.\n    每次选出连接 V_A 和 V-V_A 中顶点的权值最小的边, 把这条边加入到 MST 中.\n    并且把这条边在 V-V_A 中的顶点加到 V_A 中.\n    这样重复|V|-1 次, 找出|V|-1 条符合上述迭代方法的边, 就构成了最小生成树.\n\nPrim 算法实现一: \n    见下面 Prim1 函数.\n    设两个数组 min_wei 和 pred, 其中:\n    min_wei[u]=w 表示 V_A 中的结点到 u 的最短边权值, 其中 u 属于 V-V_A\n    pred[u]=v 表示上述 min_wei[u]=w 取到的时候, V_A 中与 u 相连的结点.\n\n    每次通过排序, 维护 min_wei, pred, 进而找出权值最小的, 横跨 V_A, V-V_A 的边.\n    把这条边(v, u) (v 属于 V_A, u 属于 V-V_A)中的 u 加入到 V_A 中, 然后如此迭代.\n    并且把这条边加入 MST.\n\n    如此重复|V|-1 次, 可以得到 MST.\n\n    复杂度估计:\n        主要复杂度花在找最小权值的横跨 V_A, V-V_A 的边上了. 复杂度大致在 O(|V|^3)左右?\n        (prim1 函数的 for 循环中嵌入一个 update, update 函数大概是 O(|V|^2))\n'

In [117]:
def build_adjM(vertexNum:int, edges:list[tuple[int, int, int]])->np.ndarray:
    vertex_id=[i for i in range(vertexNum)]
    adjM=np.full(shape=(vertexNum, vertexNum), fill_value=np.inf, dtype=np.float64)

    # 不用numpy库实现 adjM 创建: adjM=[[0 for _ in range(numCourses)] for _ in range(numCourses)]

    for (u, v, weight) in edges:
        adjM[u, v]=adjM[v, u]=weight # 无向有权图

    return adjM

In [118]:
def Prim1(adjM:np.ndarray)->tuple[set[tuple[int, int]], int]:
    verNum=adjM.shape[0]

    selected=set()

    min_wei=np.full(shape=verNum, fill_value=np.inf, dtype=np.float64) 
    # tip1: min_wei[u]=w 表示V_A中的顶点到V-V_A中的顶点的最小边权值
    # tip2: np.inf 不和 np.int16 兼容, 应当换成 dtype=np.float64

    pred=[-1 for _ in range(verNum)] #  min_wei[u]=w 对应的前驱结点

    minumum_spanning_tree=set()
    mst_weight=0

    selected.add(0)
    pred[0]=-1

    def update(): # 更新此时最小的轻边以及对应的最小生成树中的边
        for u in range(verNum):
            if not u in selected:
                for v in selected:
                    if adjM[v, u]<min_wei[u]:
                        min_wei[u]=adjM[v, u]
                        pred[u]=v

    
    for _ in range(verNum-1):
        update()
        min_weight=np.inf
        u_key=0
        for u in range(verNum):
            if not u in selected:
                if min_wei[u]<min_weight:
                    min_weight=min_wei[u]
                    u_key=u
        minumum_spanning_tree.add((pred[u_key], u_key))
        mst_weight+=adjM[pred[u_key], u_key]
        selected.add(u_key)
        # print(selected)
    
    # print(adjM)
    # print(pred)
    # print(min_wei)

    return minumum_spanning_tree, mst_weight

![Test Pic 1](MST_test1.png)

In [119]:
if __name__=='__main__':
    # 测试例子(如上图)来自 PPT P61, 结点字符已经转换成了对应的 id 编号在结点旁边
    vertexNum=9
    edges=[(0, 1, 4), (0, 4, 8), (1, 4, 1), (1, 2, 8), (2, 3, 7),
           (3, 8, 9), (7, 8, 10), (6, 7, 2), (4, 6, 1), (4, 5, 7),
           (2, 5, 2), (5, 6, 4), (2, 7, 4), (3, 7, 14)]
    
    adjM=build_adjM(vertexNum, edges)
    minimum_spanning_tree, mst_weight=Prim1(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_weight)

MST includes edges: {(0, 1), (3, 8), (4, 6), (1, 4), (2, 3), (6, 7), (7, 2), (2, 5)}
MST weight: 30.0


![Test Pic 2](MST_test2.png)

In [120]:
if __name__=='__main__':
    # 以上例子来源: 数据结构-第九课-P63.
    # 图中结点已编号在旁边
    # 最小生成树如图中红色的边所示
    
    vertexNum=6
    edges=[(0, 2, 19), (0, 3, 20), (2, 3, 22), (0, 1, 16), (1, 3, 11),
           (3, 4, 14), (2, 4, 18), (1, 4, 6), (1, 5, 5), (4, 5, 9)]
    
    adjM=build_adjM(vertexNum, edges)
    minimum_spanning_tree, mst_weight=Prim1(adjM)
    print('MST includes edges:', minimum_spanning_tree)
    print('MST weight:', mst_weight)

MST includes edges: {(0, 1), (1, 5), (4, 2), (1, 4), (1, 3)}
MST weight: 56.0
